In [1]:
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
root = '/content/gdrive/My Drive/ml project'
csv_path = os.path.join(root, 'hatespeech_text_abusive.xlsx')
df = pd.read_excel(csv_path, header = None)
df.rename(columns={0:'tweet', 1:'label'}, inplace=True)
print(df.head(10))

my_labels = ['hateful', 'normal']
len(df)

                                               tweet    label
0  RT @Papapishu: Man it would fucking rule if we...  hateful
1  It is time to draw close to Him &#128591;&#127...   normal
2  if you notice me start to act different or dis...   normal
3  Forget unfollowers, I believe in growing. 7 ne...   normal
4  RT @Vitiligoprince: Hate Being sexually Frustr...  hateful
5  Topped the group in TGP Disc Jam Season 2! Ont...   normal
6  That daily baby aspirin for your #heart just m...   normal
7  I liked a @YouTube video from @mattshea https:...   normal
8  RT @LestuhGang_: If your fucking up &amp; your...  hateful
9  @Move_Fwd give up. You've lost. You will not c...   normal


85966

In [9]:
import nltk
nltk.download('stopwords')
nltk.download('words')
from nltk.corpus import stopwords
import re
import string

STOPWORDS = stopwords.words('english')
STOPWORDS.append("rt")

WORDS = set(nltk.corpus.words.words())
WORDS.update({"fuck", "fucking"})

def clean_text():
    df["tweet"] = df["tweet"].apply(lambda x: x.lower())
    df["tweet"] = [re.sub('(@[^\s]+)|(#[^\s]+)', '', tweet) for tweet in df["tweet"]]
    df["tweet"] = [re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',tweet) for tweet in df["tweet"]]
    df["tweet"] = [re.sub('(\'[^\s]+)|(&[^\s]+)','',tweet) for tweet in df["tweet"]]
    df["tweet"] = [re.sub('[^\w\s/:%.,_-]','',tweet) for tweet in df["tweet"]]
    df["tweet"] = df["tweet"].apply(lambda tweet: tweet.translate(str.maketrans('', '', string.punctuation)))
    df["tweet"] = df["tweet"].apply(lambda tweet: tweet.translate(str.maketrans('', '', "0123456789❤♀️♥⚽️《")) )
    df["tweet"] = df["tweet"].str.split(' ').apply(lambda tweet: ' '.join(k for k in tweet if k not in STOPWORDS))
    df["tweet"] = df["tweet"].str.split(' ').apply(lambda tweet: ' '.join(k for k in tweet if k in WORDS))
    df["tweet"] = df["tweet"].str.replace(' +', ' ', case=False)
    df["tweet"] = df["tweet"].str.strip()
    df["tweet"].replace('', np.nan, inplace=True)
    df.dropna(subset=["tweet"], inplace=True)
      
clean_text()
    
print(df.head(10))
len(df)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
                                               tweet    label
0     man would fucking rule party perpetual warfare  hateful
1            time draw close father draw near always   normal
2  notice start act different distant something n...   normal
3            forget believe growing new last day via   normal
4           hate sexually like fuck ion fuck anybody  hateful
5                  topped group disc jam season onto   normal
6                     daily baby aspirin might colon   normal
7             video blue army coming ancient warfare   normal
8                          fucking dont tell fucking  hateful
9        give lost convince one iota read conspiracy   normal


84865

In [0]:
train_size = int(len(df) * .7)
train_posts = df['tweet'][:train_size]
train_tags = df['label'][:train_size]

test_posts = df['tweet'][train_size:]
test_tags = df['label'][train_size:]

max_words = 1000
tokenize = text.Tokenizer(num_words=max_words, char_level=False)
tokenize.fit_on_texts(train_posts) # only fit on train

x_train = tokenize.texts_to_matrix(train_posts)
x_test = tokenize.texts_to_matrix(test_posts)

encoder = LabelEncoder()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)

num_classes = np.max(y_train) + 1
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

In [11]:
print (len(x_train),len(x_test))
print ('Normal  tweet count in X_train:', train_posts[train_tags == "normal"].count())
print ('Hateful tweet count in X_train:', train_posts[train_tags == "hateful"].count())
print ('Normal  tweet count in X_test:', test_posts[test_tags == "normal"].count())
print ('Hateful tweet count in X_test:', test_posts[test_tags == "hateful"].count())

59405 25460
Normal  tweet count in X_train: 37465
Hateful tweet count in X_train: 21940
Normal  tweet count in X_test: 15975
Hateful tweet count in X_test: 9485


In [12]:
# Build the model
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               512512    
_________________________________________________________________
activation_1 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 1026      
_________________________________________________________________
activation_2 (Activation)    (None, 2)                 0         
Total params: 513,538
Trainable params: 513,538
Non-trainable params: 0
_______________

In [26]:
batch_size = 512
epochs = 30
              
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Train on 53464 samples, validate on 5941 samples
Epoch 1/30
53464/53464 [==============================] - 4s 75us/step - loss: 0.0624 - acc: 0.9769 - val_loss: 0.5704 - val_acc: 0.8953
Epoch 2/30
53464/53464 [==============================] - 4s 76us/step - loss: 0.0613 - acc: 0.9775 - val_loss: 0.5758 - val_acc: 0.8967
Epoch 3/30
53464/53464 [==============================] - 4s 76us/step - loss: 0.0614 - acc: 0.9768 - val_loss: 0.5814 - val_acc: 0.8970
Epoch 4/30
53464/53464 [==============================] - 4s 76us/step - loss: 0.0613 - acc: 0.9769 - val_loss: 0.5855 - val_acc: 0.8975
Epoch 5/30
53464/53464 [==============================] - 4s 76us/step - loss: 0.0616 - acc: 0.9769 - val_loss: 0.5791 - val_acc: 0.8960
Epoch 6/30
53464/53464 [==============================] - 4s 76us/step - loss: 0.0610 - acc: 0.9766 - val_loss: 0.5866 - val_acc: 0.8977
Epoch 7/30
53464/53464 [==============================] - 4s 76us/step - loss: 0.0605 - acc: 0.9777 - val_loss: 0.5868 - val_acc:

In [27]:
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test accuracy:', score[1])
print(score)

25460/25460 [==============================] - 1s 29us/step
Test accuracy: 0.8929693635392171
[0.6249159745316007, 0.8929693635392171]


In [21]:
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report

y_pred = model.predict(x_test)

#y_pred = y_test
print (y_test)
#print (pred)

for i in range(len(y_test)):
    if(y_pred[i][0] > y_pred[i][1]):
        y_pred[i][0] = 1
        y_pred[i][1] = 0
    else:
        y_pred[i][0] = 0
        y_pred[i][1] = 1

print (y_pred)

from sklearn import metrics

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred, target_names = my_labels))

[[1. 0.]
 [0. 1.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[1. 0.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
accuracy 0.893401413982718
              precision    recall  f1-score   support

     hateful       0.89      0.81      0.85      9485
      normal       0.89      0.94      0.92     15975

   micro avg       0.89      0.89      0.89     25460
   macro avg       0.89      0.88      0.88     25460
weighted avg       0.89      0.89      0.89     25460
 samples avg       0.89      0.89      0.89     25460

